In [18]:
pip install fastapi

In [19]:
pip install uvicorn

In [20]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import sklearn
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, recall_score, precision_score
import fastapi
from typing import Optional

In [21]:


_x = [1]
_y = [1]
_prediction = ["True"]
_result = ["True"]
model = DummyClassifier(strategy="constant", constant="True")
model.fit(np.array([_x,_y]).reshape(1, -1), _result)


DummyClassifier(constant='True', strategy='constant')

In [22]:
def model_evaluation(y_test, y_pred) :
    print('Trained Model Test Data Accuracy Score :',accuracy_score(y_test, y_pred)*100)
    testacc=accuracy_score(y_test, y_pred)
    testrecall=recall_score(y_test, y_pred, pos_label = 'True')
    testprecision=precision_score(y_test, y_pred, pos_label = 'True')
    print(' ')
    print(classification_report(y_test, y_pred))
    return testacc, testrecall, testprecision

In [23]:
def create_eval_df(x, y, result) :
    data = {"x": x, "y": y, "result": result}
    df = pd.DataFrame(data)
    eval_df = df.copy()
    eval_df["predict"] = model.predict(data[x,y])
    eval_df["correct"] = eval_df["predict"] == eval_df["result"]
    eval_df["correctandtrue"] = eval_df["predict"].isin([True]) & eval_df["result"].isin([True])
    return eval_df

In [24]:
app = fastapi.FastAPI()


@app.get("/predict/")
def predict(id:int = 0 , x:float =0, y:float=0):
    print(id, x, y)
    _x.insert(id, x)
    _y.insert(id, y)
    prediction = model.predict(np.array([x,y]).reshape(1, -1))
    _prediction.insert(id, prediction[0])
    print(prediction[0])
    return dict([("prediction", prediction[0])])

@app.get("/result/")
def result(id:int, result:str):
    print(result)
    _result.insert(id, result)
    return dict([("message", "Copied result for {}".format(id))])

@app.get("/retrain/")
def retrain(last_n:int):
    print ("RETRAINING over last {}".format(last_n))
    #print(_x)
    #print(_y)
    n = min(last_n, len(_x) - 1)

    # Get the arrays for training
    X = np.array([_x, _y]).T
    Y = np.array(_result)


    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40)

    print(X_train)
    print(y_train)

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    #print(X, Y)
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.40)

    print(X_train)
    print(y_train)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    testacc, testrecall, testprecision = model_evaluation(y_test, y_pred)

    return {"message": "Model retrained", "accuracy": testacc, "recall": testrecall, "precision": testprecision}

@app.get("/new_model/")
def new_model(model:str, last_n:int, strategy:Optional[str] = None, k:Optional[int] = 0):
    print(model, last_n, strategy, k)
    if model == "Dummy" :
        model = DummyClassifier(strategy=strategy)
    elif model == "KNN" :
        model = KNeighborsClassifier(k=k)
    #model.set_params(model_params)

    message = retrain(last_n=last_n)

    returns = {"message": "Model trained {}".format(model), "train": message}
    print(returns)

    return returns

#@app.get("/evaluate_model/")
#def evaluate_model(last_n:int):
#    return {"message": "Item created", "last_n": last_n}

In [26]:
#print(predict(1, 1, 1))
#result(1, True)
# Populate with synthetic data (add this before calling new_model)
for i in range(50):
    _x.append(i)
    _y.append(i % 2)
    _result.append(i % 3 == 0)
new_model("Dummy", 50, "stratified")

Dummy 50 stratified 0
RETRAINING over last 50
[[40  0]
 [31  1]
 [17  1]
 [22  0]
 [34  0]
 [47  1]
 [ 7  1]
 [45  1]
 [21  1]
 [38  0]
 [ 3  1]
 [ 5  1]
 [43  1]
 [42  0]
 [44  0]
 [28  0]
 [36  0]
 [ 9  1]
 [11  1]
 [ 2  0]
 [16  0]
 [14  0]
 [46  0]
 [37  1]
 [27  1]
 [20  0]
 [49  1]
 [15  1]
 [23  1]
 [32  0]]
['False' 'False' 'False' 'False' 'False' 'False' 'False' 'True' 'True'
 'False' 'True' 'False' 'False' 'True' 'False' 'False' 'True' 'True'
 'False' 'False' 'False' 'False' 'False' 'False' 'True' 'False' 'False'
 'True' 'False' 'False']
[[49  1]
 [18  0]
 [24  0]
 [33  1]
 [41  1]
 [19  1]
 [ 1  1]
 [44  0]
 [39  1]
 [45  1]
 [31  1]
 [14  0]
 [48  0]
 [23  1]
 [10  0]
 [42  0]
 [11  1]
 [46  0]
 [ 9  1]
 [ 5  1]
 [15  1]
 [ 1  1]
 [ 3  1]
 [17  1]
 [21  1]
 [ 4  0]
 [ 0  0]
 [12  0]
 [43  1]
 [13  1]]
['False' 'True' 'True' 'True' 'False' 'False' 'True' 'False' 'True' 'True'
 'False' 'False' 'True' 'False' 'False' 'True' 'False' 'False' 'True'
 'False' 'True' 'False' 'True'

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'message': "Model trained DummyClassifier(strategy='stratified')",
 'train': {'message': 'Model retrained',
  'accuracy': 0.19047619047619047,
  'recall': 1.0,
  'precision': 0.19047619047619047}}

In [27]:
@app.get("/items/")
async def read_item(skip: int = 0, limit: int = 10):
    return {"skip": skip, "limit": limit}

In [28]:
import asyncio
import uvicorn

if __name__ == "__main__":
    config = uvicorn.Config(app)
    server = uvicorn.Server(config)
    loop = asyncio.get_running_loop()
    loop.create_task(server.serve())